In [5]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

In [2]:
##############################################################################
# Fitness Function
##############################################################################
def fitness_function(xy_tuple):
    x = xy_tuple[0] / 200
    y = xy_tuple[1] / 200
    return np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

In [4]:
##############################################################################
# Grey Wolf Optimization Algorithm
##############################################################################
def grey_wolf_optimization(search_space, num_wolves, max_iterations):
    """
    Implements the Grey Wolf Optimization algorithm to find the maximum
    of a given fitness function within a specified search space.

    Args:
        search_space (list of tuples): A list defining the lower and upper bounds
                                       for each dimension. For a 1000x1000 area,
                                       it would be [(-1000, 1000), (-1000, 1000)].
        num_wolves (int): The number of grey wolves in the population.
        max_iterations (int): The maximum number of iterations to run the algorithm.

    Returns:
        tuple: A tuple containing the coordinates of the best solution found
               and its corresponding fitness value.
    """
    num_dimensions = len(search_space)
    wolves_positions = np.random.uniform(low=[s[0] for s in search_space],
                                         high=[s[1] for s in search_space],
                                         size=(num_wolves, num_dimensions))
    wolves_fitness = np.array([fitness_function(pos) for pos in wolves_positions])

    # Initialize alpha, beta, and delta wolves
    alpha_index = np.argmax(wolves_fitness)
    alpha_position = wolves_positions[alpha_index].copy()
    alpha_fitness = wolves_fitness[alpha_index]

    # Initialize beta and delta with very low fitness (for maximization)
    beta_index = -1
    beta_position = np.zeros(num_dimensions)
    beta_fitness = -float('inf')

    delta_index = -1
    delta_position = np.zeros(num_dimensions)
    delta_fitness = -float('inf')

    # Identify beta and delta
    for i in range(num_wolves):
        if i != alpha_index:
            if wolves_fitness[i] > beta_fitness:
                beta_fitness = wolves_fitness[i]
                beta_position = wolves_positions[i].copy()
                beta_index = i
            elif wolves_fitness[i] > delta_fitness and i != beta_index:
                delta_fitness = wolves_fitness[i]
                delta_position = wolves_positions[i].copy()
                delta_index = i

    convergence_curve = []

    # Main loop of GWO
    for iteration in range(max_iterations):
        a = 2 - iteration * (2 / max_iterations)  # a decreases linearly from 2 to 0

        for i in range(num_wolves):
            r1 = np.random.rand(num_dimensions)
            r2 = np.random.rand(num_dimensions)

            A1 = 2 * a * r1 - a
            C1 = 2 * r2
            D_alpha = np.abs(C1 * alpha_position - wolves_positions[i])
            X1 = alpha_position - A1 * D_alpha

            r1 = np.random.rand(num_dimensions)
            r2 = np.random.rand(num_dimensions)

            A2 = 2 * a * r1 - a
            C2 = 2 * r2
            D_beta = np.abs(C2 * beta_position - wolves_positions[i])
            X2 = beta_position - A2 * D_beta

            r1 = np.random.rand(num_dimensions)
            r2 = np.random.rand(num_dimensions)

            A3 = 2 * a * r1 - a
            C3 = 2 * r2
            D_delta = np.abs(C3 * delta_position - wolves_positions[i])
            X3 = delta_position - A3 * D_delta

            wolves_positions[i] = np.clip((X1 + X2 + X3) / 3,
                                          [s[0] for s in search_space],
                                          [s[1] for s in search_space])

        # Update fitness values
        wolves_fitness = np.array([fitness_function(pos) for pos in wolves_positions])

        # Update alpha, beta, and delta
        best_fitness_current_iteration = np.max(wolves_fitness)
        best_position_current_iteration = wolves_positions[np.argmax(wolves_fitness)]

        if best_fitness_current_iteration > alpha_fitness:
            alpha_fitness = best_fitness_current_iteration
            alpha_position = best_position_current_iteration.copy()

        # Update beta and delta (ensuring they are different from alpha)
        beta_fitness = -float('inf')
        beta_index = -1
        delta_fitness = -float('inf')
        delta_index = -1

        for i in range(num_wolves):
            if i != alpha_index:
                if wolves_fitness[i] > beta_fitness:
                    beta_fitness = wolves_fitness[i]
                    beta_position = wolves_positions[i].copy()
                    beta_index = i
                elif wolves_fitness[i] > delta_fitness and i != beta_index:
                    delta_fitness = wolves_fitness[i]
                    delta_position = wolves_positions[i].copy()
                    delta_index = i

        convergence_curve.append(alpha_fitness)

    return alpha_position, alpha_fitness, convergence_curve



In [6]:
##############################################################################
# Set parameters and run the optimization
##############################################################################
search_space = [(0, 1000), (0, 1000)]
num_wolves = 50
max_iterations = 100

best_position, best_fitness, convergence_curve = grey_wolf_optimization(
    search_space, num_wolves, max_iterations
)

print("Best Position (x, y):", best_position)
print("Best Fitness Value:", best_fitness)

# You can also plot the convergence curve to see how the fitness improved over iterations
# plt.plot(convergence_curve)
# plt.xlabel("Iteration")
# plt.ylabel("Best Fitness Value")
# plt.title("Convergence Curve of GWO")
# plt.grid(True)
# plt.show()

Best Position (x, y): [627.87969868 628.37420058]
Best Fitness Value: 0.9999926622992031
